In [ ]:
import mlflow
import pandas as pd
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import mlflow.sklearn
from mlflow.models.signature import infer_signature

In [ ]:
mlflow.set_tracking_uri('http://localhost:5000')

os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'password'
os.environ['AWS_DEFAULT_REGION'] = 'us-east-1'
endpoint = 'http://localhost:9000'
os.environ['MLFLOW_S3_ENDPOINT_URL'] = endpoint

In [ ]:
model_name = "my-registered-model"

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names) 

iris_train = iris_df.iloc[:-1]
iris_test = iris_df.iloc[-1:]

experiment_id = mlflow.set_experiment('model-demo')
with mlflow.start_run():
    clf = RandomForestClassifier(max_depth=7, random_state=0)
    clf.fit(iris_train, iris.target[:-1])
    signature = infer_signature(iris_train, clf.predict(iris_train))
    mlflow.sklearn.log_model(clf, "iris_rf", signature=signature, registered_model_name=model_name)

In [ ]:
import mlflow.pyfunc

model_version = 1

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)

result = model.predict(iris_test)

print(result)

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

stage = 'Staging'

client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage
)

model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{stage}"
)

result = model.predict(iris_test)

print(result)

In [ ]:
from pprint import pprint

for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)

In [ ]:
client.delete_registered_model(name=model_name)

In [ ]:
for rm in client.list_registered_models():
    pprint(dict(rm), indent=4)